In [1]:
from pathlib import Path
import pandas as pd

In [23]:
LLAMA_RESPONSE = "llama_response"
QUESTION = "question"
RESOLUTION_CRITERIA = "resolution_criteria"
LLAMA_RESPONSE = "llama_response"
OUTPUT_DIR = Path().resolve() / "output"
CLEANED_OUTPUT_DIR = Path().resolve() / "cleaned_output"
GPT35_CSV = Path().resolve().parent / "inference" / "output" / "GPT-3.5_sampled_test_set_responses.csv"
assert OUTPUT_DIR.exists()
assert GPT35_CSV.exists()
CLEANED_OUTPUT_DIR.mkdir(exist_ok=True)

In [3]:
output_files = list(OUTPUT_DIR.glob("*.csv"))

In [15]:
OUTPUT_FILE_INDEX = 0
df_llama1 = pd.read_csv(output_files[OUTPUT_FILE_INDEX])

In [5]:
df_llama1

,question,resolution_criteria,general_prompt,llama_response
0,Sam Altman criminal charges by Nov 30?,Not applicable/available for this question.,Here is a forecasting question: Sam Altman cri...,Here is a forecasting question: Sam Altman cri...
1,Will Prigozhin make an appearance by Friday?,Not applicable/available for this question.,Here is a forecasting question: Will Prigozhin...,Here is a forecasting question: Will Prigozhin...
2,Will xQc beat Forsen's Minecraft record by the...,Not applicable/available for this question.,Here is a forecasting question: Will xQc beat ...,Here is a forecasting question: Will xQc beat ...
3,NFL Week 12 (Nov 26): Will the Philadelphia Ea...,Not applicable/available for this question.,Here is a forecasting question: NFL Week 12 (N...,Here is a forecasting question: NFL Week 12 (N...
4,Will General Sergei Surovikin be stripped of h...,The question will resolve Yes if there are off...,Here is a forecasting question: Will General S...,Here is a forecasting question: Will General S...
5,"NBA (Oct 25, 2023): Will the Indiana Pacers be...",Not applicable/available for this question.,"Here is a forecasting question: NBA (Oct 25, 2...","Here is a forecasting question: NBA (Oct 25, 2..."
6,Will the next speaker be elected by Oct 10?,Not applicable/available for this question.,Here is a forecasting question: Will the next ...,Here is a forecasting question: Will the next ...
7,Will 'Barbie' gross over $16m this weekend?,Not applicable/available for this question.,Here is a forecasting question: Will 'Barbie' ...,Here is a forecasting question: Will 'Barbie' ...
8,Will Kane sign for Bayern Munich?,Not applicable/available for this question.,Here is a forecasting question: Will Kane sign...,Here is a forecasting question: Will Kane sign...
9,Will Adam22 and Lena The Plug break up this mo...,Not applicable/available for this question.,Here is a forecasting question: Will Adam22 an...,Here is a forecasting question: Will Adam22 an...


In [6]:
first_row = df_llama1.loc[0]
first_row

question                          Sam Altman criminal charges by Nov 30?
resolution_criteria          Not applicable/available for this question.
general_prompt         Here is a forecasting question: Sam Altman cri...
llama_response         Here is a forecasting question: Sam Altman cri...
Name: 0, dtype: object

In [7]:
template = """Here is a forecasting question: {question}\n\nHere are the resolution criteria of the question: {resolution_criteria}.\n\nBased on the forecasting question and resolution criteria, generate step-by-step reasoning to create a resolution. Output exactly 6 steps without an introduction or a conclusion."""  # noqa: E501

In [8]:
def left_strip(q: str, rc: str, resp: str) -> str:
    """Strip the prompt from the LlaMA response."""
    prefix = template.format(question=q, resolution_criteria=rc)
    return resp.replace(prefix, "")

In [17]:
df_llama1[LLAMA_RESPONSE] = df_llama1.apply(lambda row: left_strip(row[QUESTION], row[RESOLUTION_CRITERIA], row[LLAMA_RESPONSE]), axis=1)

In [31]:
df_llama1.to_csv(CLEANED_OUTPUT_DIR / ("cleaned_" + output_files[OUTPUT_FILE_INDEX].name), index=False)